# 環境設定

configファイルにパスを通す。パスの参照順は以下の通り。
Jupyterから特定のconfigファイルを読ませるために、カレントフォルダにrtc.confを事前に作成しておく。

* コマンドラインオプション "-f"
* 環境変数 "RTC_MANAGER_CONFIG"
* デフォルト設定ファイル "./rtc.conf"
* デフォルト設定ファイル "/etc/rtc.conf"
* デフォルト設定ファイル "/etc/rtc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc/rtc.conf"
* 埋め込みコンフィギュレーション値

In [1]:
!ls ./rtc.conf

./rtc.conf


rtc.confの内容は以下の通り。
logger.enableをYESに設定し起動履歴をログに出力する。Python上ではうまくエラーが拾えないので必須。

logger.file_nameで、ログ・ファイル名を指定。rtc.log に出力する設定とする。

In [6]:
!more ./rtc.conf

corba.nameservers: localhost
exec_cxt.periodic.rate:1000.0
naming.formats: %n.rtc
logger.enable: YES
logger.file_name: rtc.log


# RTCのソース

今回は、104のAngleをSPI通信で、SP3008というチップを経由して取得しにいくので、`import spidev`でspiのライブラリをimportする。　

In [1]:
#!/usr/bin/env python
# coding: utf-8
# Python 

import RPi.GPIO as GPIO
import spidev
import time
import sys
import RTC
import OpenRTM_aist

configurationの値を設定する。

conf.default.angle_pin は、システム・ダイアグラムのconfigurationタブから任意の値に変更が可能。

In [2]:
LED_PIN = 4

faboled_spec = ["implementation_id", "FaBoLED",
              "type_name",         "FaBoLEDComponent",
              "description",       "FaBo LED component",
              "version",           "1.0",
              "vendor",            "GClue, Inc.",
              "category",          "fabo",
              "activity_type",     "DataFlowComponent",
              "max_instance",      "10",
              "language",          "Python",
              "lang_type",         "script",
              ""]

LEDの点滅をループ

In [3]:
class FaBoLED(OpenRTM_aist.DataFlowComponentBase):

    def __init__(self, manager):
        print "__init__"
        OpenRTM_aist.DataFlowComponentBase.__init__(self, manager)

        self._ledFlag     = RTC.TimedBoolean(RTC.Time(0,0),0)
        self._ledFlagIn     = OpenRTM_aist.InPort("LED", self._ledFlag)
        self.addInPort("LED", self._ledFlagIn)

        return

    def onInitialize(self):
        print "onInitialize"

        GPIO.setwarnings(False)
        GPIO.setmode( GPIO.BCM )
        GPIO.setup( LED_PIN, GPIO.OUT )
        self.toggle = True
        return RTC.RTC_OK

    def onExecute(self, ec_id):
        print "onExcute"
        self.toggle  = self._ledFlagIn.read()
        GPIO.output( LED_PIN, self.toggle.data)
        time.sleep(0.1)

        return RTC.RTC_OK

In [ ]:
def FaBoLEDInit(manager):
    print "FaBoLEDInit"
    profile = OpenRTM_aist.Properties(defaults_str=faboled_spec)
    manager.registerFactory(profile,
                            FaBoLED,
                            OpenRTM_aist.Delete)    

def MyModuleInit(manager):
    print "MyModuleInit"
    FaBoLEDInit(manager)

    # Create a component
    comp = manager.createComponent("FaBoLED")

def main():
    print "main"
    mgr = OpenRTM_aist.Manager.init(sys.argv)
    mgr.setModuleInitProc(MyModuleInit)
    mgr.activateManager()
    mgr.runManager()

if __name__ == "__main__":
    main()

main
MyModuleInit
FaBoLEDInit
__init__
onInitialize
onExcute
